# 🦜🔗 LangChain Fundamentals with Groq API

Welcome to **LangChain Fundamentals**! This notebook will teach you the core concepts of LangChain using the **free Groq API**.

## What You'll Learn:
1. **Setup & Configuration** - Environment setup with Groq API
2. **ChatGroq Model** - Initialize and use Groq's LLM
3. **Messages** - SystemMessage, HumanMessage, AIMessage
4. **Prompt Templates** - Reusable prompts with variables
5. **LCEL Chains** - Combine components with the `|` operator
6. **Output Parsers** - Parse and structure LLM responses
7. **Streaming & Batching** - Efficient response handling
8. **Tools** - Function calling with LangChain

---

## Prerequisites
- Get your free Groq API key at: https://console.groq.com/keys
- Add it to your `.env` file as `GROQ_API_KEY=your_key_here`

## 1. Setup & Installation

First, let's install and import the required packages.

In [ ]:
# Install required packages (uncomment if needed)
# %pip install langchain langchain-groq python-dotenv -q

In [1]:
# Import required libraries
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# If .env doesn't work, you can set the API key directly:
# os.environ["GROQ_API_KEY"] = "your-groq-api-key-here"

# Verify API key is loaded
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    print("⚠️ API Key not found! Please either:")
    print("   1. Create a .env file with: GROQ_API_KEY=your-key-here")
    print("   2. Or uncomment and set the os.environ line above")
else:
    print(f"✅ API Key loaded successfully!")

✅ API Key loaded successfully!


## 2. Initialize ChatGroq Model

**ChatGroq** is LangChain's integration with Groq's fast inference API. 

### Available Free Groq Models:
| Model | Speed | Best For |
|-------|-------|----------|
| `llama-3.3-70b-versatile` | 280 tok/s | General purpose, high quality |
| `llama-3.1-8b-instant` | 560 tok/s | Fast responses, simpler tasks |
| `mixtral-8x7b-32768` | 450 tok/s | Complex reasoning |

In [3]:
from langchain_groq import ChatGroq
llm =ChatGroq(
    model='llama-3.3-70b-versatile',
    temperature=0,
    max_tokens=1024,
    max_retries=2,
    
)
print(f" Model init",{llm.model_name})

 Model init {'llama-3.3-70b-versatile'}


In [7]:
response=llm.invoke("What is langchain?,write in one sentence")


In [8]:
print(response.content)

LangChain is an open-source framework that enables developers to build applications powered by large language models, providing a set of tools and libraries to interact with and fine-tune these models.


## 3. Messages in LangChain

LangChain uses **message types** to structure conversations:

- **SystemMessage**: Sets the behavior/role of the AI
- **HumanMessage**: User's input
- **AIMessage**: AI's response (for conversation history)

In [10]:
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage

messages=[
    SystemMessage(content="You are a helpful python tutor,Keep explanation simple and use examples"),
    HumanMessage(content="What is difference between list and tuple?")
]
response=llm.invoke(messages)
print(response.content)

**Lists vs Tuples**

In Python, `lists` and `tuples` are two types of data structures that can store multiple values. The main difference between them is:

* **Immutability**: Tuples are **immutable**, meaning you cannot change their values after creation. Lists are **mutable**, meaning you can change their values after creation.
* **Syntax**: Tuples use parentheses `()` and lists use square brackets `[]`.

**Example:**
```python
# Tuple (immutable)
my_tuple = (1, 2, 3)
print(my_tuple)  # Output: (1, 2, 3)
# my_tuple[0] = 10  # This will raise an error

# List (mutable)
my_list = [1, 2, 3]
print(my_list)  # Output: [1, 2, 3]
my_list[0] = 10
print(my_list)  # Output: [10, 2, 3]
```
When to use each:

* Use **tuples** when you need to store a collection of values that shouldn't be changed, like a record or a constant.
* Use **lists** when you need to store a collection of values that may need to be changed, like a dynamic array or a queue.


In [12]:
conversation=[
    SystemMessage(content="You are a friendly AI assistant"),
    HumanMessage(content="My name is Rahul"),
    AIMessage(content="Nice to meet you,Rahul! How can I hep you?"),
    HumanMessage(content="What is my Name?")
]
response=llm.invoke(conversation)
print(response.content)

Your name is Rahul.


## 4. Prompt Templates

**Prompt Templates** let you create reusable prompts with variables. This is essential for building scalable LLM applications.

In [14]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_messages([
    ("system","You are expert in {topic}.Provide clear and consice answer"),
    ("human","{question}"),
])
format_prompt=prompt.invoke({
    "topic":"Gen AI",
    "question":"What is one Major advantage of Transformer model?"
}
)
print("Formatted messages")
for msg in format_prompt.messages:
    print(f"{msg.type}: {msg.content}")

Formatted messages
system: You are expert in Gen AI.Provide clear and consice answer
human: What is one Major advantage of Transformer model?


In [15]:
response=llm.invoke(format_prompt)
print(response.content)

**Parallelization**: One major advantage of the Transformer model is its ability to be parallelized, allowing for much faster training times compared to traditional recurrent neural networks (RNNs).


In [16]:
format_prompt=prompt.invoke({
    "topic":"Health science",
    "question":"What is advantage of running "
}
)
print("Formatted messages")
for msg in format_prompt.messages:
    print(f"{msg.type}: {msg.content}")

Formatted messages
system: You are expert in Health science.Provide clear and consice answer
human: What is advantage of running 


In [21]:
response=llm.invoke(format_prompt)
print(response.content)

The advantages of running include:

1. **Improves Cardiovascular Health**: Running strengthens the heart and lungs, improving circulation and reducing the risk of heart disease.
2. **Weight Management**: Running helps burn calories and maintain a healthy weight.
3. **Increases Mental Well-being**: Running releases endorphins, also known as "feel-good" hormones, which can help reduce stress and anxiety.
4. **Boosts Immune System**: Running can help stimulate the immune system, reducing the risk of illness and infection.
5. **Improves Sleep**: Regular running can help improve sleep quality and duration.
6. **Increases Bone Density**: Running can help strengthen bones, reducing the risk of osteoporosis and fractures.
7. **Reduces Risk of Chronic Diseases**: Running can help reduce the risk of chronic diseases, such as diabetes, certain types of cancer, and stroke.
8. **Improves Mental Clarity and Focus**: Running can help improve cognitive function and mental clarity.
9. **Increases Energ

## 5. LCEL Chains (LangChain Expression Language)

**LCEL** is LangChain's way to compose components using the **pipe operator `|`**. It's like Unix pipes for LLM applications!

```
prompt | llm | output_parser
```

This creates a chain where data flows: `input → prompt → llm → output_parser → output`

In [ ]:
from langchain_core.output_parsers import StrOutputParser

prompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant.Be concise to your answer"),
    ("human","Explain {concept} in simple terms")
])

chain=prompt | llm | StrOutputParser()
result=chain.invoke({"concept":"neural networks"})
print(result)

Neural networks are like a computer brain that learns from data. They're made of layers of "neurons" that process information, similar to how our brain works. 

1. Input layer: Receives data (e.g., images, text)
2. Hidden layers: Process and analyze the data
3. Output layer: Produces a result (e.g., prediction, classification)

The network adjusts itself to improve accuracy, allowing it to learn and make decisions like a human.


In [22]:
topics=["blockchain","API","recurssion"]
for topic in topics:
    result=chain.invoke({"concept":topic})
    print(result[:100])

Blockchain is a digital ledger that records transactions across a network of computers. It's like a 
API (Application Programming Interface) is like a messenger between systems. It helps different appl
**Recursion**: A process where a function calls itself repeatedly until it reaches a stopping point.


## 6. Output Parsers - Structured Output

Output parsers transform LLM text responses into structured data like **JSON** or **Pydantic models**.

In [24]:
from pydantic import BaseModel,Field
from typing import List
class MovieReccomondation(BaseModel):
    """A movie recommendation with details"""
    title:str=Field(description="The movie title")
    year: int = Field(description="Release year")
    genre: str = Field(description="Primary genre")
    reason: str = Field(description="Why this movie is recommended")


stuructured_llm=llm.with_structured_output(MovieReccomondation)

response=stuructured_llm.invoke("Recommend any thriller Indian movie from 2010s")
print(f"Title: {response.title}")
print(f"Year: {response.year}")
print(f"Genre: {response.genre}")
print(f"Reason: {response.reason}")



Title: Drishyam
Year: 2015
Genre: Thriller
Reason: It is a well-crafted thriller with a gripping storyline and strong performances.


## 7. Streaming & Batching

- **Streaming**: Get responses token-by-token (great for UX)
- **Batching**: Process multiple inputs in parallel (great for efficiency)

In [26]:
print("Streaming response:")
for chunk in llm.stream("Write a joke to a chess player"):
    print(chunk.content)



Streaming response:

Why
 did
 the
 chess
 player
 bring
 a
 ladder
 to
 the
 tournament
?


Because
 they
 wanted
 to
 take
 their
 game
 to
 the
 next
 level
!
 (
get
 it
?)




In [29]:
prompts=[
    "what is Python?",
    "What is Javascript?",
    "What is Rust?",
]

responses=llm.batch(prompts)
# print(responses)
for prompt ,response in zip(prompts,responses):
    print(f"Prompt:{prompt}")
    print(f"Response:{response.content[:100]}...\n")

Prompt:what is Python?
Response:**Python** is a high-level, interpreted programming language that is widely used for various purpose...

Prompt:What is Javascript?
Response:**JavaScript** is a high-level, dynamic, and interpreted programming language that is primarily used...

Prompt:What is Rust?
Response:**Rust** is a systems programming language that prioritizes safety, performance, and concurrency. It...



## 8. Tools & Function Calling

**Tools** allow the LLM to call external functions. This is the foundation for building **AI agents**.

In [48]:
from langchain_core.tools import tool

@tool
def get_weather(city:str)->str:
    """ Get the current weather of the city """
    weather_data={
        "new delhi":"Sunny ,30degree Cel",
        "mumbai":"Sunnay 29 degree Cel",
    }
    return weather_data.get(city.lower(),f"Weather data is not available for {city}")


@tool
def calculate(expresssion:str)->str:
    """Evaluate a mathematical operation"""
    try:
        result=eval(expresssion)
        return f"Result: {result}"
    except Exception as e:
        print("Exception :{e}")

llm_with_tool=llm.bind_tools([get_weather,calculate])

In [50]:
# Ask the LLM something that requires a tool
response = llm_with_tool.invoke("What's the weather in New Delhi?")

# Check if the LLM wants to call a tool
if response.tool_calls:
    print("🔧 LLM wants to call tools:")
    for tool_call in response.tool_calls:
        print(f"  Tool: {tool_call['name']}")
        print(f"  Args: {tool_call['args']}")
        
        # Execute the tool
        if tool_call['name'] == 'get_weather':
            result = get_weather.invoke(tool_call['args'])
            print(f"  Result: {result}")
else:
    print("LLM responded directly:", response.content)

🔧 LLM wants to call tools:
  Tool: get_weather
  Args: {'city': 'New Delhi'}
  Result: Sunny ,30degree Cel


In [53]:
# Ask the LLM something that requires a tool
response = llm_with_tool.invoke("What is 2+3*5 +2?")

# Check if the LLM wants to call a tool
if response.tool_calls:
    print("🔧 LLM wants to call tools:")
    for tool_call in response.tool_calls:
        print(f"  Tool: {tool_call['name']}")
        print(f"  Args: {tool_call['args']}")
        
        # Execute the tool
        if tool_call['name'] == 'get_weather':
            result = get_weather.invoke(tool_call['args'])
            print(f"  Result: {result}")
        if tool_call['name'] == 'calculate':
            result = calculate.invoke(tool_call['args'])
            print(f"  Result: {result}")
else:
    print("LLM responded directly:", response.content)

🔧 LLM wants to call tools:
  Tool: calculate
  Args: {'expresssion': '2+3*5 +2'}
  Result: Result: 19


In [54]:
# Ask the LLM something that requires a tool
response = llm_with_tool.invoke("What is newton third law?")

# Check if the LLM wants to call a tool
if response.tool_calls:
    print("🔧 LLM wants to call tools:")
    for tool_call in response.tool_calls:
        print(f"  Tool: {tool_call['name']}")
        print(f"  Args: {tool_call['args']}")
        
        # Execute the tool
        if tool_call['name'] == 'get_weather':
            result = get_weather.invoke(tool_call['args'])
            print(f"  Result: {result}")
        if tool_call['name'] == 'calculate':
            result = calculate.invoke(tool_call['args'])
            print(f"  Result: {result}")
else:
    print("LLM responded directly:", response.content)

LLM responded directly: Newton's third law, also known as the law of reciprocal actions, states that for every action, there is an equal and opposite reaction. This means that when an object exerts a force on another object, the second object will exert an equal and opposite force on the first object. In other words, forces always occur in pairs, and these pairs are equal in magnitude and opposite in direction.


## 🎯 Summary

In this notebook, you learned the **LangChain Fundamentals**:

| Concept | What You Learned |
|---------|------------------|
| **ChatGroq** | Initialize Groq models with temperature, max_tokens |
| **Messages** | SystemMessage, HumanMessage, AIMessage for conversations |
| **Prompt Templates** | Create reusable prompts with variables |
| **LCEL Chains** | Compose components with the `\|` operator |
| **Output Parsers** | Get structured data using Pydantic models |
| **Streaming** | Token-by-token response for better UX |
| **Batching** | Process multiple inputs efficiently |
| **Tools** | Let LLMs call external functions |

---

### Next Steps: RAG Fundamentals
In the next notebook, we'll cover **Retrieval Augmented Generation (RAG)**:
- Document loaders
- Text splitters  
- Embeddings
- Vector stores
- Retrieval chains